In [1]:
import datetime
import pandas as pd
import numpy as np

# Load the raw data (all months and years)
df = pd.read_csv('../data/flights.csv', low_memory=False)

# --- Create proper datetime columns ---
# Convert date components to datetime
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

# Add day of week (1=Monday, 7=Sunday)
df['DAY_OF_WEEK'] = df['DATE'].dt.dayofweek + 1

# --- Select and rename columns ---
columns_needed = [
    'DATE', 'DAY_OF_WEEK',
    'FLIGHT_NUMBER', 'TAIL_NUMBER', 
    'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
    'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
    'SCHEDULED_TIME', 'WHEELS_OFF', 'WHEELS_ON',
    'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY',
    'CANCELLATION_REASON', 'DISTANCE'
]
df = df[columns_needed]

# --- Parse time fields ---
def parse_time(val):
    if pd.isnull(val):
        return None, None
    val = int(val)
    if val == 2400:  # Handle midnight special case
        val = 0
    hour = val // 100
    minute = val % 100
    if hour < 0 or hour > 23 or minute < 0 or minute > 59:
        return None, None
    return hour, minute

# Process all time fields
time_fields = {
    'SCHEDULED_DEPARTURE': ['SCHED_DEP_HOUR', 'SCHED_DEP_MIN'],
    'DEPARTURE_TIME': ['DEP_HOUR', 'DEP_MIN'],
    'SCHEDULED_ARRIVAL': ['SCHED_ARR_HOUR', 'SCHED_ARR_MIN'],
    'ARRIVAL_TIME': ['ARR_HOUR', 'ARR_MIN'],
    'WHEELS_OFF': ['WHEELS_OFF_HOUR', 'WHEELS_OFF_MIN'],
    'WHEELS_ON': ['WHEELS_ON_HOUR', 'WHEELS_ON_MIN']
}

for original, (hour_col, min_col) in time_fields.items():
    df[hour_col], df[min_col] = zip(*df[original].apply(parse_time))

# --- Clean up ---
# Remove original time columns
df = df.drop(list(time_fields.keys()), axis=1)

# Remove rows with missing essential data
essential_columns = [
    'DATE', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
    'SCHED_DEP_HOUR', 'SCHED_DEP_MIN',
    'SCHED_ARR_HOUR', 'SCHED_ARR_MIN',
    'ARRIVAL_DELAY'  # target variable
]
df = df.dropna(subset=essential_columns)

# Add month and year columns back (might be useful for analysis)
df['MONTH'] = df['DATE'].dt.month
df['YEAR'] = df['DATE'].dt.year

# Reorder columns logically
final_columns = [
    'YEAR', 'MONTH', 'DATE', 'DAY_OF_WEEK',
    'FLIGHT_NUMBER', 'TAIL_NUMBER', 'AIRLINE',
    'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DISTANCE',
    'SCHED_DEP_HOUR', 'SCHED_DEP_MIN', 'DEP_HOUR', 'DEP_MIN', 'DEPARTURE_DELAY',
    'SCHED_ARR_HOUR', 'SCHED_ARR_MIN', 'ARR_HOUR', 'ARR_MIN', 'ARRIVAL_DELAY',
    'WHEELS_OFF_HOUR', 'WHEELS_OFF_MIN', 'WHEELS_ON_HOUR', 'WHEELS_ON_MIN',
    'SCHEDULED_TIME', 'CANCELLATION_REASON'
]
df = df[final_columns]

# Save cleaned data
df.to_csv('cleaned_flights.csv', index=False)
print("Cleaned data saved with shape:", df.shape)
print("Date range:", df['DATE'].min(), "to", df['DATE'].max())

Cleaned data saved with shape: (5714008, 26)
Date range: 2015-01-01 00:00:00 to 2015-12-31 00:00:00


In [2]:
df

,YEAR,MONTH,DATE,DAY_OF_WEEK,FLIGHT_NUMBER,TAIL_NUMBER,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,...,SCHED_ARR_MIN,ARR_HOUR,ARR_MIN,ARRIVAL_DELAY,WHEELS_OFF_HOUR,WHEELS_OFF_MIN,WHEELS_ON_HOUR,WHEELS_ON_MIN,SCHEDULED_TIME,CANCELLATION_REASON
0,2015,1,2015-01-01,4,98,N407AS,AS,ANC,SEA,1448,...,30,4.0,8.0,-22.0,0.0,15.0,4.0,4.0,205.0,NaN
1,2015,1,2015-01-01,4,2336,N3KUAA,AA,LAX,PBI,2330,...,50,7.0,41.0,-9.0,0.0,14.0,7.0,37.0,280.0,NaN
2,2015,1,2015-01-01,4,840,N171US,US,SFO,CLT,2296,...,6,8.0,11.0,5.0,0.0,34.0,8.0,0.0,286.0,NaN
3,2015,1,2015-01-01,4,258,N3HYAA,AA,LAX,MIA,2342,...,5,7.0,56.0,-9.0,0.0,30.0,7.0,48.0,285.0,NaN
4,2015,1,2015-01-01,4,135,N527AS,AS,SEA,ANC,1448,...,20,2.0,59.0,-21.0,0.0,35.0,2.0,54.0,235.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2015,12,2015-12-31,4,688,N657JB,B6,LAX,BOS,2611,...,19,7.0,53.0,-26.0,0.0,17.0,7.0,49.0,320.0,NaN
5819075,2015,12,2015-12-31,4,745,N828JB,B6,JFK,PSE,1617,...,46,4.0,30.0,-16.0,0.0,12.0,4.0,27.0,227.0,NaN
5819076,2015,12,2015-12-31,4,1503,N913JB,B6,JFK,SJU,1598,...,40,4.0,32.0,-8.0,0.0,7.0,4.0,24.0,221.0,NaN
5819077,2015,12,2015-12-31,4,333,N527JB,B6,MCO,SJU,1189,...,40,3.0,30.0,-10.0,0.0,3.0,3.0,27.0,161.0,NaN
